# Optimization algorithms

## Mini-batch gradient descent

將資料分組如每 64, 128, 256, ... 個資料為一組，每一組記為 $ \big( X^{\{i\}}, Y^{\{i\}} \big) $

如 m = 5,000,000 個資料，mini-batch size = 1000 每組一千個資料，

- `X.shape = ( nx, m )`
- `Y.shape = (  1, m )`
- $ X^{\{i\}}$ `.shape = ( nx, 1000 ) `
- $ Y^{\{i\}}$ `.shape = (  1, 1000 ) `
- i = 1 ~ 5000

One epoch of mini-batch GD:

$
\text{for t = 1 ... 5000} \\
\ \ \ \text{# forward prop on } X^{\{t\}} \text{ for 1000 examples} \\
\ \ \ Z^{\{t\}} = W^{\{t\}} X^{\{t\}} + b^{\{t\}} \\
\ \ \ A^{\{t\}} = g^{\{t\}}\big( Z^{\{t\}} \big) \\
\ \ \ \text{# Compute cost } \\
\ \ \ J^{\{t\}} = \frac{1}{1000} \sum_{i=1}^l \mathcal{L} \big( \hat{y}^{(i)}, y^{(i)} \big) +
\frac{\lambda}{2 \cdot 1000} \sum_l \Vert W^{\{t\}} \Vert_F^2 \\
\ \ \ \text{# back prop on } J^{\{t\}} \\
\ \ \ W^{[l]} = W^{[l]} - \alpha \cdot dW^{[l]} \\
\ \ \ b^{[l]} = b^{[l]} - \alpha \cdot db^{[l]} \\
$

在 Batch GD 的一個 epoch 只能做 一次 GD，用了 mini-batch 可以做 5000 次 GD.

## Exponentially Weighted (moving) Averages

每組資料 $ (\theta_1, \theta_2, \theta_3, \dots ) $ 取前 約10組的平均:

$
v_0 = 0 \\
v_1 = 0.9 v_0 + 0.1 \theta_1 \\
v_2 = 0.9 v_0 + 0.1 \theta_2 \\
\vdots \\
v_t = 0.9 v_{t-1} + 0.1 \theta_t
$

$$
v_t = \beta v_{t-1} + \big( 1 - \beta \big) \theta_t \\
$$

如果 $ \theta_t $ 是每天氣溫，$ v_t $ 大約是前 $ \frac{1}{1-\beta} $ 天的平均氣溫。

$ \beta = 0.9 $, 就大約是前 10 天的平均。  
$ \beta = 0.98 $, 就大約是前 50 天的平均。

Bias correction

在剛啟動時，前面幾筆會發生均值過低的問題，可以用 Bias correction 修正:

$$
v_t = \frac{\beta v_{t-1} + \big( 1 - \beta \big) \theta_t}{1 - \beta^t}
$$

## GD with Momentum

在進行 mini-batch 時候，GD下降可能會偏移最佳方向，不停地震盪。希望能夠在 "正確"的方向維度上放大進度，"錯誤"的方向維度上縮小進度。

這裏用 w 代表"正確"的方向維度，b 代表"錯誤"的方向維度。

Hyperparameters:

- $ \alpha $ : learning rate
- $ \beta_1 $ : controller of Exponentially Weighted Average, 通常為 0.9

On iteration t, compute dw, db on current mini-batch

$$
v_{dW} = \beta_1 \ v_{dW} + \big( 1 - \beta_1 \big) dW \\
v_{db} = \beta_1 \ v_{db} + \big( 1 - \beta_1 \big) db \\
W = W - \alpha \ v_{dW} \\
b = b - \alpha \ v_{db}
$$

通常無需做 bias correction, 因為在 10 個 iteration 後幾乎已經沒影響。

另一種版本: $ v_{dW} = \beta \ v_{dW} + dW $

## RMSprop (Root Mean Square)

Hyperparameters:

- $ \beta_2 $ : controller of Exponentially Weighted Average in RMSprop
- $ \epsilon = 10^{-8} $ : 避免除數為零。

$$
S_{dW} = \beta_2 \ S_{dW} + \big( 1 - \beta_2 \big) dW^2 \\
S_{db} = \beta_2 \ S_{db} + \big( 1 - \beta_2 \big) db^2 \\
W := W - \alpha \ \frac{dw}{\sqrt{S_{dW}} + \epsilon} \\
b := b - \alpha \ \frac{db}{\sqrt{S_{db}} + \epsilon}
$$

## Adam optimization

ADAptive Moment estimation

Momentum + RMSprop, 要加上 bias correction

$
\beta_1 = 0.9 \to \ \ dW \text{... (moment)} \\
\beta_2 = 0.999 \to \ \ dW^2 \text{... (second moment)} \\
\epsilon = 10^{-8}
$

update parameters using both RMSprop + Momentum

$$
v_{W^{[l]}} = \beta_1 v_{W^{[l]}} + (1 - \beta_1) \frac{\partial J }{ \partial W^{[l]} } \\
v^{corrected}_{W^{[l]}} = \frac{v_{W^{[l]}}}{1 - (\beta_1)^t} \\
s_{W^{[l]}} = \beta_2 s_{W^{[l]}} + (1 - \beta_2) (\frac{\partial J }{\partial W^{[l]} })^2 \\
s^{corrected}_{W^{[l]}} = \frac{s_{W^{[l]}}}{1 - (\beta_2)^t} \\
W^{[l]} = W^{[l]} - \alpha \frac{v^{corrected}_{W^{[l]}}}{\sqrt{s^{corrected}_{W^{[l]}}}+\varepsilon}
$$


## Learning Rate Decay

hyperparameters:

- $ \alpha_0 $
- decay_rate

$$
\alpha = \frac{1}{1 + \text{ decay_rate * epoch}} \alpha_0
$$


if $ \alpha_0 = 0.2 $, decay_rate=1


|epoch|alpha|
|-|-|
| 1 | 0.1 |
| 2 | 0.067 |
| 3 | 0.05 |
| 4 | 0.04 |
| 5 | 0.033 |
| 6 | 0.029 |

Other learning rate decay methods:

$
\alpha = 0.95^{\text{epoch}} \ \alpha_0 \\
\alpha = \frac{k}{\sqrt{\text{epoch}}} \ \alpha_0 \\
\alpha = \frac{k}{\sqrt{\text{t}}} \ \alpha_0 \\
$

## Local Optima &amp; Saddle Point

絕大部分的 slope=0 的點，不是 Local Optima, 而是 Saddle Point

在低維度的空間中，每個維度都是 convex 會形成 Local Optima,  
但在 高維度的空間中，如 $ n_x=20000 $，要每個維度都是 convex 的機率實在很低。因此部份維度是 convex, 只是 saddle point.

## Program Skeleton

```
parameters = initialize_parameters()
v          = initialize_velocity # for momentum
v, s       = initialize_adam     # for adam ( momentum + RMSprop )

for i in range(num_epochs)
   minibatches = random_mini_batches(X, Y, mini_batch_size, random_seed)
   
   for minibatch in minibatches
      a3, caches = forward_propagation(minibatch_X, parameters)
      cost       = compute_cost(a3, minibatch_Y)
      grads      = backward_propagation(minibatch_X, minibatch_Y, caches)
      parameters = update_parameters(...)
```